## 导入库

In [18]:
import random
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

In [3]:
SEED = 42
torch.manual_seed(SEED)
# 每次运行网络的时候算法和SEED是固定的，方便复现
torch.backends.cudnn.deterministic = True

##  数据预处理

In [7]:
TEXT = data.Field(tokenize='spacy', tokenizer_language='en_core_web_sm')
LABEL = data.LabelField(dtype=torch.float)

In [8]:
train, test = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


D:\Jupyter\NLP_Learning\Sentiment-Analysis\.data\imdb\aclImdb_v1.tar.gz: 100%|████| 84.1M/84.1M [00:09<00:00, 9.26MB/s]


In [9]:
len(train), len(test)

(25000, 25000)

In [15]:
len(vars(train.examples[0])['text'])

165